In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sc
import csv
import plotly as py
import sklearn as skl
from apyori import apriori
import gc
from orangecontrib.associate.fpgrowth import *
%matplotlib inline

In [206]:
''' All of the code below which is commented out is used to create the stateKeepRows .csv file. Comment
    out if the stateKeepRows.csv file is not available. Will take > 2 hours to process '''

' All of the code below which is commented out is used to create the stateKeepRows .csv file. Comment\n    out if the stateKeepRows.csv file is not available. Will take > 2 hours to process '

In [3]:
### Select only the STORE column from trnsact
#trnsactCol = ["SKU", "STORE", "REGISTER", "TRANNUM", "INTERID", "SALEDATE", "STYPE", "QUANTITY", "AMT", "ORGPRICE", "ORGPRICE2", "SEQ", "MIC", "?"]
#trnsactSTORE = pd.read_csv("D:\\DillardsPOS\\trnsact.csv", sep=',', header=None, names=trnsactCol, usecols=["STORE"])

In [4]:
### import entire strinfo table
#strinfoCol = ["CITY", "STATE", "ZIP", "?"]
#strinfo = pd.read_csv('D:\\DillardsPOS\\strinfo.csv', sep=',', engine='python', header=None, names=strinfoCol)

In [5]:
### create array stateSelect with has 1's for each transaction made in a Texas store
#stateSelect = np.zeros(len(trnsactSTORE))
#ARSTORE = [2,4,204,304,400,404,504,604,704,804,1000,1304,2400,3400,6400,8006,8106,8206,8306,8406,8506,8706,8808,8906,9800,9806,9900,9906]
#for i in np.arange(len(trnsactSTORE)):
#    if trnsactSTORE.STORE[i] in ARSTORE:
#        stateSelect[i] = 1

In [6]:
### read in state select csv
#stateSelect = pd.read_csv("D:\\DillardsPOS\\stateSelect.csv", sep=',', header=None, names=["Arkansas?"])

In [7]:
### convert stateSelect to an int array
#stateSelectInt = stateSelect["Arkansas?"].astype(dtype=int)

In [8]:
### calculate array containing indices of trnsact rows we want to skip
#stateSkipRows = [i for i, x in enumerate(stateSelectInt) if x == 0]

In [9]:
### array of rows we want to keep (Arkansas stores)
#stateKeepRows = [i for i, x in enumerate(stateSelectInt) if x == 1]

In [10]:
### store in csv for future use
#np.savetxt("D:\\DillardsPOS\\stateKeepRows.csv", stateKeepRows, delimiter=',')

In [12]:
stateKeepRows = pd.read_csv("D:\\DillardsPOS\\stateKeepRows.csv", header=None, names=["stateKeepRows"])
stateSkipRows = np.arange(120916896)
stateSkipRows = np.delete(stateSkipRows, obj=stateKeepRows, axis=0)

In [13]:
# import only Arkansas transactions to trnsactAR dataframe
trnsactCol = ["SKU", "STORE", "REGISTER", "TRANNUM", "INTERID", "SALEDATE", "STYPE", "QUANTITY", "AMT", "ORGPRICE", "ORGPRICE2", "SEQ", "MIC", "?"]
primaryKeys = ["SKU", "STORE", "REGISTER", "TRANNUM", "SALEDATE", "STYPE", "SEQ"]
trnsactAR = pd.read_csv("D:\\DillardsPOS\\trnsact.csv", sep=',', header=None, names=trnsactCol, skiprows=stateSkipRows, usecols=primaryKeys)

In [14]:
# delete return transactions
trnsactAR = trnsactAR[trnsactAR.STYPE == "P"]

In [15]:
# groups each individual item transaction into a basket
grouped = trnsactAR["SKU"].groupby([trnsactAR["STORE"], trnsactAR["REGISTER"], trnsactAR["TRANNUM"], trnsactAR["SALEDATE"]])
groups = grouped.groups

In [102]:
# list of lists baskets holds all of the market baskets
baskets = []

for name, group in grouped:
    baskets.append(grouped.get_group((name)).values)
    #baskets.append(groups[name]["SKU"])

In [151]:
itemSets = dict(frequent_itemsets(baskets, 250))
associationRules = list(association_rules(itemsets, .077))

In [179]:
lH = np.zeros(50)
for i in np.arange(len(associationRules)):
    lH[i] = next(iter(associationRules[i][0]))
lH = lH.astype(int)
    
rH = np.zeros(50)
for i in np.arange(len(associationRules)):
    rH[i] = next(iter(associationRules[i][1]))
rH = rH.astype(int)
    
support = np.zeros(50)
for i in np.arange(len(associationRules)):
    support[i] = associationRules[i][2]
support = support.astype(int)
    
confidence = np.zeros(50)
for i in np.arange(len(associationRules)):
    confidence[i] = round(associationRules[i][3], 4)

In [204]:
d = {'Left Hand' : lH,
     'Right Hand' : rH,
     'Support' : support,
     'Confidence' : confidence}
displayTable = pd.DataFrame(d)
displayTable = displayTable[['Left Hand', 'Right Hand', 'Support', 'Confidence']]
displayTable = displayTable.sort_values(by='Confidence', axis=0, ascending=False)
displayTable = displayTable.set_index(np.arange(1,51,1))

In [205]:
displayTable

,Left Hand,Right Hand,Support,Confidence
1,6062521,6072521,625,0.7070
2,6072521,6062521,625,0.6779
3,6032521,6072521,549,0.6169
4,6072521,6032521,549,0.5954
5,6062521,6032521,516,0.5837
6,6032521,6062521,516,0.5798
7,3988011,2716578,680,0.2951
8,3968011,3898011,846,0.2902
9,3908011,3988011,528,0.2763
10,3908011,3998011,512,0.2679


In [ ]:
''' The following code was used in data exploration to import the different tables in the database '''

In [ ]:
# import entire deptinfo table
#deptinfoCol = ["DEPT", "DEPTDESC", "?"]
#deptinfo = pd.read_csv('D:\\DillardsPOS\\deptinfo.csv', sep=',', header=None, names=deptinfoCol, engine='python')

In [ ]:
# import entire skstinfo table
#skstinfoCol = ["SKU", "STORE", "COST", "RETAIL", "?"]
#skstinfo = pd.read_csv('D:\\DillardsPOS\\skstinfo.csv', sep=',', header=None, names=skstinfoCol, engine='python')

In [ ]:
# import entire skuinfo table
#skuinfoCol = ["SKU", "CLASSID", "DEPT", "UPC", "STYLE", "COLOR", "SIZE", "PACKSIZE", "VENDOR", "BRAND"]
#skuinfo = pd.read_csv('D:\\DillardsPOS\\skuinfo.csv', sep=',', engine='python', header=None, names=skuinfoCol, usecols=skuinfoCol)

In [ ]:
# import entire sample of trnsact table
#trnsactCol = ["SKU", "STORE", "REGISTER", "TRANNUM", "INTERID", "SALEDATE", "STYPE", "QUANTITY", "AMT", "ORGPRICE", "ORGPRICE2", "SEQ", "MIC", "?"]
#trnsact = pd.read_csv("D:\\DillardsPOS\\trnsact.csv", sep=',', header=None, names=trnsactCol, engine='python', nrows=10000)

In [ ]:
# Code selects only the STYPE column from trnsact
#trnsactCol = ["SKU", "STORE", "REGISTER", "TRANNUM", "INTERID", "SALEDATE", "STYPE", "QUANTITY", "AMT", "ORGPRICE", "ORGPRICE2", "SEQ", "MIC", "?"]
#trnsactSTYPE = pd.read_csv("D:\\DillardsPOS\\trnsact.csv", sep=',', header=None, names=trnsactCol, usecols=["STYPE"])

In [ ]:
# Helper code for data exploration
#unique, counts = np.unique((trnsact["ORGPRICE"] == trnsact["ORGPRICE2"]), return_counts=True)
#print(unique)